# Panda IK

## Sliders

In [8]:
import numpy as np
from functools import partial

from ipywidgets import FloatSlider, Layout, Dropdown
from IPython.display import display, HTML, SVG

from pydrake.common.jupyter import process_ipywidget_events
from pydrake.multibody.tree import JointIndex
from pydrake.systems.framework import BasicVector, VectorSystem

import pydrake.multibody as multibody
from pydrake.solvers.mathematicalprogram import MathematicalProgram
from pydrake.solvers.mathematicalprogram import Solve

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)

from pydrake.math import RotationMatrix
from pydrake.math import RollPitchYaw

In [16]:
def MakeIKSlidersThatPublishOnCallback(
    plant, publishing_system, root_context, end_link_name, B, pos_tol = 0.02, theta_tol = 0.01, my_callback = None, 
        lower_limit = [0,0,0,0,0,0], upper_limit = [1,1,1,2*np.pi,2*np.pi,2*np.pi], resolution = 0.001, length = 400, 
        continuous_update = True):
    """
    Creates an ipywidget slider for the desired position (TODO: and orientation) of hand link of the plant.
    It updates the context and calls Publish directly from slider callback. 
    This function is based on pydrake/multibody/jupyter_widgets.py
    
    Args:
        plant: A MultibodyPlant
        
        publishing system: The System whos publish method will be called. Can be the entire Diagram, or just a subsystem
        
        root_context: A mutable root Context of the Diagram containing the ``plant`` and the ``publishing_system``; we will extract 
        the subcontext's using `GetMyContextFromRoot`. 
        
        end_link_name: the name of the end_effector link
        
        B: the position relative to the end effector link that we are trying to get within the bounding box.
        
        pos_tol: the width of the bounding box tolerance around where the ik problem tries to find a solution
        
        theta_tol: the tolerance between the desired orientation and the achieved orientation
        
        my_callback: An optional additional callback function that will be called once immediately and again whenever 
        the sliders are moved. 
        
        lower_limit: A numpy array of length 6 defining the lower limit of the (x,y,z, roll, pitch, yaw) sliders.
        
        upper_limit: A numpy array of length 6 defining the upper limit of the (x,y,z, roll, pitch, yaw) sliders.
        
        resolution: A scalar that specifies the step argument of the FloatSlider.
        
        length: The length of the sliders, which will be passed as a string to CSS width field and can be any valid CSS
        entry (e.g. 100, 100 px).
        
        continuous_update: The continuous update field for the FloatSliders. The default ``True`` means that this method
        will publish/callback as the sliders are dragged. ``False`` means that the publish/callback will only happen
        once the user finishes dragging the slider.
        
    Returns:
        A list of slider wiget objects that are created.
        
    Note that some publishers (like MeshcatVisualizer) use an initialization event to "load" the geometry. You should call
    that before this method (e.g. with `meshcat.load()`).
    """
    
    def _broadcast(x, num):
        """
        Args: scalar ``x``, numpy array of length ``num``
        Return: a numpy array of length ``num`` storing x
        """
        x = np.array(x)
        assert len(x.shape) <= 1
        return np.array(x)*np.ones(num)
    
    num_sliders = 6
    
    # make arrays for the slider parameters
 
    resolution = _broadcast(resolution, num_sliders)
    desired_pose = np.array([0.5,0.5,0.5, 0,0,0])
    

    
    publishing_context = publishing_system.GetMyContextFromRoot(root_context) # Returns the mutable subsystem context for this system, given a root context.
    plant_context = plant.GetMyContextFromRoot(root_context)
    positions = plant.GetPositions(plant_context)
    print('starting joint angles:', positions)
    
    # publish once immediately
    publishing_system.Publish(publishing_context)
    if my_callback:
        my_callback(plant_context)
        
    
    def ik_solver():
        # create the bounding box where we want the end-effector
        p_AQ_lower = desired_pose[0:3] - pos_tol*0.5 
        p_AQ_upper = desired_pose[0:3] + pos_tol*0.5
        
        # instantiate an IK problem
        ik_problem = multibody.inverse_kinematics.InverseKinematics(plant, plant_context)

        # add a position constraint to the IK problem (where the hand needs to be)
        ik_problem.AddPositionConstraint(plant.GetFrameByName(end_link_name), B, plant.world_frame(), p_AQ_lower,p_AQ_upper)
        
        R_AbarA = RotationMatrix(RollPitchYaw(np.array([0,0,0])))
        R_BbarB = RotationMatrix(RollPitchYay(desired_pose[3:]))
        
        ik_problem.AddOrientationConstraint(plant.GetFrameByName(end_link_name), R_AbarA, plant.world_frame(), R_BbarB, theta_tol)
        
        # get and solve the problem
        prog = ik_problem.prog()
        result = Solve(prog)
        

        
    def _slider_callback(change, index):
        desired_pose[index] = change.new
        
        result = ik_solver()
      
        plant.SetPositions(plant_context, result.GetSolution())
        publishing_system.Publish(publishing_context)
        if my_callback:
            my_callback(plant_context)
    
    slider_widgets = []
    names = ['x (m)', 'y (m)', 'z (m)', 'roll (rad)', 'pitch (rad)', 'yaw (rad)']

    print(desired_pose)
    for i in range(num_sliders):
        slider = FloatSlider(
            value = desired_pose[i],
            min = lower_limit[i],
            max = upper_limit[i],
            step = resolution[i],
            continuous_update = continuous_update,
            description = names[i],
            style = {'description_width': 'initial'},
            layour = Layout(width = f"'{length}'")
        )
        slider.observe(partial(_slider_callback, index = i), names = 'value')
        display(slider)
        slider_widgets.append(slider)
        
    return slider_widgets
    
    
    
    

### Start Meshcat Server

In [12]:
server_args = []
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)


In [17]:
panda_file = FindResourceOrThrow("drake/manipulation/models/franka_description/urdf/panda_arm_hand.urdf")

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
Parser(plant, scene_graph).AddModelFromFile(panda_file)
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("panda_link0"))
plant.Finalize()

meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)
diagram = builder.Build()
context = diagram.CreateDefaultContext()

meshcat.load()
MakeIKSlidersThatPublishOnCallback(plant, meshcat, context, "panda_hand", np.array([0,0,0]), continuous_update = True);

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6016...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7016/static/
Connected to meshcat-server.
starting joint angles: [0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.5 0.5 0.5 0.  0.  0. ]


In [10]:
L = np.array([1,2,3,4,5,6])
print(L[0:3])
print(L[3:])

[1 2 3]
[4 5 6]
